In [513]:
import os
import json
import pandas as pd
import pickle
import numpy as np
import time
from legiscan import LegiScan
from legiscan import LegiScanError
from datetime import datetime
import regex as re 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

with open('./assets/imp/API_keys', 'r') as file:
    config = json.load(file)

# Extract the API key
legi = config['legiscan']
legis = LegiScan(legi)

#for state year, we store our sampled left and right bill ids from textalignments.ipynb. Will use this in read json
with open('./Data/left_dict.pkl', 'rb') as pickle_file:
    left_dict = pickle.load(pickle_file)

with open('./Data/left_dict.pkl.', 'rb') as pickle_file:
    right_dict = pickle.load(pickle_file)

key_left = [key for key in left_dict.keys()]
key_right = [key for key in right_dict.keys()]

In [723]:
def read_json_and_count(file_path):
    ''' Read the JSON file line by line and if bill text exists then append state and year to list.
     Then we group by state and list. The outpout is used in visualization of state and year wise number of document available '''
    # List to store all bill information with categories
    all_bills = []
    start = time.time()

    with open(file_path, 'r', encoding='utf-8') as file:
        line_count = 0  # Initialize line counter
        for line in file:
            try:
                bill = json.loads(line.strip())
                state = bill.get('state')
                date_created = bill.get('date_created')
                year = datetime.strptime(date_created, "%Y-%m-%d %H:%M:%S").year if date_created else 'NA'
                has_first = bill.get('bill_document_first') is not None
                has_last = bill.get('bill_document_last') is not None
                
                # Determine category of document availability
                if has_first or has_last:
                    years = year
                elif not has_first and not has_last:
                    years = 'na'
  # Set year as 'NA' if both parts are missing

                # Add bill information to the list
                all_bills.append({'state': state, 'year': years})

                line_count += 1
                if line_count % 500 == 0:  # Print progress every 500 lines
                    print(f"Processed {line_count} lines.")
                    end = time.time()
                    time_taken = (end - start) / 60
                    print(f'Time taken to process 500 lines: {time_taken} minutes')
                    start = time.time()
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

    # Convert list of bills into DataFrame
    all_bills_df = pd.DataFrame(all_bills)

    # Group by 'state', 'year', and 'document_status' and count occurrences
    detailed_counts = all_bills_df.groupby(['state', 'year']).size().reset_index(name='count')

    return detailed_counts

In [724]:
###get state and yearwise counts 
counts = read_json_and_sample(file_path)
counts = counts.groupby(['state','year']).count()
# counts.to_csv('./Data/counts.csv')

Processed 500 lines.
Time taken to process 500 lines: 0.012935888767242432 minutes
Processed 1000 lines.
Time taken to process 500 lines: 0.0010542710622151693 minutes
Processed 1500 lines.
Time taken to process 500 lines: 0.0008895754814147949 minutes
Processed 2000 lines.
Time taken to process 500 lines: 0.0011269132296244304 minutes
Processed 2500 lines.
Time taken to process 500 lines: 0.0009267131487528483 minutes
Processed 3000 lines.
Time taken to process 500 lines: 0.0008203625679016113 minutes
Processed 3500 lines.
Time taken to process 500 lines: 0.0014007647832234701 minutes
Processed 4000 lines.
Time taken to process 500 lines: 0.0007506251335144043 minutes
Processed 4500 lines.
Time taken to process 500 lines: 0.0009383002916971843 minutes
Processed 5000 lines.
Time taken to process 500 lines: 0.0007283767064412435 minutes
Processed 5500 lines.
Time taken to process 500 lines: 0.0006361762682596842 minutes
Processed 6000 lines.
Time taken to process 500 lines: 0.0008433818

In [441]:
def read_json_and_sample(file_path, key_left = key_left, key_right = key_right, left_dict = left_dict, right_dict = right_dict):
    ''' Read the JSON file line by line and if bill text exists and if bill id and state match from our unique bills found from
    sampling alignments data then append and convert to dataframe with all metadata.'''
    
    # Dictionary to store all bills, keyed by state
    matched_bills = []
    start = time.time()

    with open(file_path, 'r', encoding='utf-8') as file:
            line_count = 0  # Initialize line counter
            for line in file:
                try:
                    bill = json.loads(line.strip())
                    billnum = bill.get('bill_id').replace(' ', '')  # Remove whitespace

                    state = bill.get('state').lower()  # Convert state to lowercase

                    # Check if the bill_id is in key_left or key_right
                    if billnum in key_left:
                        print(billnum)
                        expected_state = left_dict.get(billnum).lower()  # Get corresponding state and convert to lowercase
                        if expected_state == state:  # Compare the states
                            matched_bills.append(bill)  # Append the bill

                    elif billnum in key_right:
                        expected_state = right_dict.get(billnum, '').lower()  # Get corresponding state and convert to lowercase
                        if expected_state == state:  # Compare the states
                            matched_bills.append(bill)  # Append the bill

                    line_count += 1
                    if line_count % 500 == 0:  # Print progress every 500 lines
                        print(f"Processed {line_count} lines.")
                        end = time.time()
                        time_taken = (end - start) / 60
                        print(f'Time taken to process 500 lines: {time_taken} minutes')
                        start = time.time()
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")

    matched_bills_df = pd.DataFrame(matched_bills)
    return matched_bills_df

file_path = './Data/state_bills.json'

In [442]:
all_bills = read_json_and_sample('./Data/state_bills.json')

Processed 500 lines.
Time taken to process 500 lines: 0.0010923107465108236 minutes
Processed 1000 lines.
Time taken to process 500 lines: 0.0007870276769002279 minutes
Processed 1500 lines.
Time taken to process 500 lines: 0.0007864753405253093 minutes
Processed 2000 lines.
Time taken to process 500 lines: 0.0008180141448974609 minutes
Processed 2500 lines.
Time taken to process 500 lines: 0.0007648944854736328 minutes
Processed 3000 lines.
Time taken to process 500 lines: 0.0007667740186055502 minutes
Processed 3500 lines.
Time taken to process 500 lines: 0.0009121974309285482 minutes
Processed 4000 lines.
Time taken to process 500 lines: 0.0007108211517333985 minutes
Processed 4500 lines.
Time taken to process 500 lines: 0.00075989564259847 minutes
Processed 5000 lines.
Time taken to process 500 lines: 0.0007753372192382812 minutes
Processed 5500 lines.
Time taken to process 500 lines: 0.0006702939669291178 minutes
Processed 6000 lines.
Time taken to process 500 lines: 0.00077075163

In [445]:
df = all_bills.copy()
df['date_created'] = pd.to_datetime(df['date_created'])

# Extract the column as a list (if needed)
date_list = df['date_created'].tolist()

# Find min and max dates
min_date = min(date_list)
max_date = max(date_list)

print("Minimum date:", min_date)
print("Maximum date:", max_date)

Minimum date: 2010-07-09 16:16:19
Maximum date: 2015-05-02 02:02:19


In [446]:
def get_names(st,text,years, name_list):
    ''' Call Legiscan API
    Conduct string matching of bill title with all results from legiscan API. Kepe only if 90% match ore more'''

    save = 0

    st = st

    if text is not None:
        ti_unproc = text
        ### clean up bill title format. topic - title - metadata. only want title
        parts = ti_unproc.split(' - ')
        if len(parts) > 1: 
            ti = parts[1].strip()  
        else:
            ti = ti_unproc  

        year_str = str(years)
        year = int(year_str[:4])
        sponsors = []
        
        bill_search = legis.search(state = st, query= ti, year= year, page=1)
        counter = 0
        for i in range(len(bill_search['results'])):
            if counter <= len(name_list):
                if bill_search['results'][i]['title'] is not None:

                    ## conducting text match  
                    list1 = bill_search['results'][i]['title'].split()  
                    list2 = ti.split()  
                    matches = sum(1 for x, y in zip(list1, list2) if x == y)
                    if len(list1) > 0:  
                        percentage_match = (matches / len(list2)) 
                    else:
                        percentage_match = 0
                    if percentage_match > 0.9:
                        bill_detail = legis.get_bill(bill_id= bill_search['results'][i]['bill_id'])
                        all_people = bill_detail['sponsors']
                        if len(all_people) >=1:
                            people_id_list = [] 
                            for j in range(len(all_people)):
                                people_id = all_people[j]['people_id']
                                name = legis.get_sponsor(people_id)
                                sponsors.append(name['name'])
                            
                            
            else: 
                break
    else: 
        sponsors = []
        
    print(sponsors)
    print('____________________________________________')
    return sponsors 

In [447]:
df_new = df.copy()

## Extract sponsors from dictionary object in column
df_new['name_list'] = df_new['sponsers'].apply(lambda sponsers: [x['name'].lower() for x in sponsers if x['type'] == 'primary'])

In [448]:
def split_dataframe(df, chunk_size):
    chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
    return chunks

# Split df_new into chunks of 100 rows each``
chunk_size = 100
df_chunks = split_dataframe(df_new, chunk_size)

In [450]:
j = 0
chunk_results = []  # This will hold the results for all chunks
error_dict = {}

for chunk in df_chunks:
    print(j)
    print('____________________________________________')
    result = []  # Reset result list for each chunk

    for i in range(len(chunk)):
        row_data = chunk.iloc[i]  # Get current row
        st = row_data['state']
        text = row_data['bill_title']
        years = row_data['date_introduced']
        name_list = row_data['name_list']

        # Debug prints
        
        print(i)
        print(st)
        print(text)
        print(years)
        print(name_list)

        try:
            ff = get_names(st, text, years, name_list) 
            result.append(ff)  
        except Exception as e:  
            print(f'Error processing row {i} in current chunk, setting results to None.')
            ## store what chunk and what row error ocurred. set row result to empty list
            error_dict[j] = i

            result = [None] * len(chunk)  
            break  
    chunk_results.append(result)  # Append the collective or None results of this chunk to chunk_results
    j += 1
    

0
____________________________________________
0
nh
relative to hearings in the department of safety.
2013-01-03 11:50:16
['jim b rausch']
[]
____________________________________________
1
nh
establishing a commission on housing policy and regulation.
2013-01-31 11:03:40
['bob odell']
[]
____________________________________________
2
nh
repealing New Hampshire's atomic energy policy.
2013-01-03 09:25:01
['robert a backus']
[]
____________________________________________
3
nh
relative to the authority of the department of state.
2013-01-03 03:22:25
['lynne m ober']
[]
____________________________________________
4
nh
relative to child passenger restraint requirements.
2013-01-03 11:30:41
['sally h kelly']
[]
____________________________________________
5
nh
relative to membership of village districts in regional planning commissions.
2013-01-03 01:25:34
['james e coffey']
[]
____________________________________________
6
nh
relative to the disposition of dedicated funds.
2015-01-08 09:0

In [451]:
## chunks where API gave some unexplaiable error, Usually related to what is being queries for row. Not clea what is issue with the query, did not find any tangible pattern

error_chunks = [key for key in error_dict.keys()]
error_rect = {}



## rules for how to slice problematic rows out of chunks. Have to keep in mind API call limit. Optimized for that
for key in error_chunks:
    a = error_dict[key]
    if (a <10):
        error_rect[key] = [slice(a+5,101)]
    elif (a<80):
        error_rect[key] = [slice(None, a-2), slice(a+2, 101)]
    elif (a>=80):
        error_rect[key] = [slice(None, a-1)]

chunk_results_errors = [] 
j = 0 
for chunk_index, chunk in enumerate(df_chunks):
    if chunk_index in error_chunks:
        print(j)
        print('____________________________________________')
        result = [None] * len(chunk)  

## try those chunks again, but subste out the problematic rows from that chunk. Problematic row information stored in dictionary during first iteration of API calls
        
        if chunk_index in error_rect:
            for row_slice in error_rect[chunk_index]:
                for i in range(*row_slice.indices(len(chunk))):  
                    row_data = chunk.iloc[i]
                    st = row_data['state']
                    text = row_data['bill_title']
                    years = row_data['date_introduced']
                    name_list = row_data['name_list']

                    # Debug prints
                    print(i)
                    print(st)
                    print(text)
                    print(years)
                    print(name_list)

                    try:
                        ff = get_names(st, text, years, name_list)
                        result[i] = ff  
                    except Exception as e:
                        print(f'Error processing row {i} in current chunk, setting results to None.')
                        result[i] = None  

        chunk_results_errors.append(result)
        j +=1


0
____________________________________________
0
sd
repeal certain provisions regarding the South Dakota Risk Pool.
2015-01-13 00:00:00
['health and human services']
Error processing row 0 in current chunk, setting results to None.
1
sd
adopt the South Dakota Nonprofit Corporation Act.
2015-01-23 00:00:00
['stevens', 'gibson', 'gosch', 'johns', 'rusch', 'tieszen']
Error processing row 1 in current chunk, setting results to None.
2
sd
repeal the South Dakota Energy Infrastructure Authority.
2015-01-13 00:00:00
['commerce and energy']
Error processing row 2 in current chunk, setting results to None.
3
sd
revise certain provisions regarding zero based budgets, budgeting procedures, and the coordination of federally aided programs.
2015-01-13 00:00:00
['appropriations']
Error processing row 3 in current chunk, setting results to None.
4
sd
revise certain provisions regarding the payment of salary and compensation to legislators.
2015-01-28 00:00:00
['gosch', 'bartling', 'gibson', 'haggar (

In [469]:
## append the chunks with no errors and errors 

k = 0
df_names = pd.DataFrame()
for j in range(0,len(df_chunks)):
    g['chunk'+str(j)] = df_chunks[j].copy()
    if j in error_chunks:
        g['chunk'+str(j)]['full_names'] = chunk_results_errors[k]
        k+=1
    else: 
        g['chunk'+str(j)]['full_names'] = chunk_results[j]

    df_names = pd.concat([df_names,g['chunk'+str(j)]])

In [456]:
## df_names.to_excel('./Data/withAPI.xlsx')

In [731]:
df_names.shape

(2585, 21)

In [510]:

### Start Manual Matching 

### first lets look at all bills with no legiscan match and have  only one sponsor  and have at least two names for sponsor
slice_full = df_names.copy()
slice_full = slice_full[(slice_full['full_names'].isna()) | (slice_full['full_names'].apply(lambda x: x == []))]
slice_full['len_list'] = slice_full['name_list'].apply(lambda x: len(x))
slice_full  = slice_full[slice_full.len_list == 1]
slice_full['len_name'] = slice_full['name_list'].apply(lambda x: len(x[0].split()))
slice_full = slice_full[slice_full.len_name >= 2]


In [511]:
def process_name(name_list):
    if not name_list:  # Check if the list is empty
        return ['', '']  # Return empty first and last names
    
    #get the full name for this slice 
    full_name = name_list[0] 

    #and split first last and possibly middle name
    names = full_name.split()
    
    # If there is a comma in the name
    if ',' in full_name:
        parts = full_name.replace(',', '').split()  # Remove comma and split
        if len(parts) >= 3:  # Last, First, Middle
            first_name = parts[1]  # Second element becomes the first name
            last_name = parts[2] + ' ' + parts[0]  # Combine third element and first element as last name. Now in first and last name format

        elif len(parts) == 2:  #last, first
            first_name = parts[1]  # Second element becomes the first name
            last_name = parts[0]  # First element becomes the last name
        else:  # Handling unexpected format gracefully
            first_name = parts[0]  # Use what we have
            last_name = ''
    else:
        # Handling names without comma
        if len(names) >= 2:  # More than two names without comma. then first name last name format
            first_name, last_name = names[0], names[-1]  # Take only first and last
        else:
            first_name, last_name = names[0], ''  # Only one name given

    return first_name, last_name
# Apply the function to each row in the DataFrame
## get two columns
slice_full[['first_name', 'last_name']] = slice_full['name_list'].apply(lambda x: process_name(x)).tolist()

In [514]:
slice_full.head(2)


## get ideological dataset
leg = pd.read_csv('C:/Users/ayush/Downloads/legislators.csv')
leg['first_name'] = leg['first_name'].str.lower()
leg['last_name'] = leg['last_name'].str.lower()

def remove_roman_numerals(s):
    ''' Remove roman numerals from names'''
    pattern = r'\b(i{1,3}|iv|vi{0,3}|ix)\b'
    return re.sub(pattern, '', s).strip()

leg['last_name'] = leg['last_name'].apply(remove_roman_numerals)

leg['house'] = leg['u_id'].dropna().apply(lambda x: str(x)[6])
leg['house'] = np.where((leg['house'] == 'L')&(leg['house'].notna()), 'lower',leg['house'])
leg['house'] = np.where((leg['house'] == 'U')&(leg['house'].notna()), 'upper',leg['house'])
leg = leg[['first_name','last_name','party','state','ideology','house']]
leg.head(2)


##merge on last name. 
look = pd.merge(slice_full, leg, on = ['last_name', 'state'], how = 'left')

,date_signed,date_introduced,bill_document_first,date_updated,short_title,bill_type,actions,summary,chamber,state,...,bill_title,sponsers,bill_id,sunlight_id,name_list,full_names,len_list,len_name,first_name,last_name
0,2013-06-27 01:12:07,2013-01-03 11:50:16,\r\n\r\n\r\nSB 0013\r\n\r\n\r\n\r\n \r\n\r\n\r...,2013-08-13 01:57:16,None,[bill],"[{'date': '2013-01-03 11:50:16', 'action': 'In...",None,upper,nh,...,relative to hearings in the department of safety.,"[{'leg_id': 'NHL000021', 'type': 'primary', 'n...",SB 13,NHB00002073,[jim b rausch],[],1,3,jim,rausch
1,2013-07-15 02:08:29,2013-01-31 11:03:40,\r\n\r\n\r\nSB 0185\r\n\r\n\r\n\r\n \r\n\r\n\r...,2013-08-29 00:38:17,None,[bill],"[{'date': '2013-01-31 11:03:40', 'action': 'In...",None,upper,nh,...,establishing a commission on housing policy an...,"[{'leg_id': 'NHL000019', 'type': 'primary', 'n...",SB 185,NHB00002765,[bob odell],[],1,2,bob,odell


,first_name,last_name,party,state,ideology,house
0,al,adams,D,ak,-1.02,lower
1,tom,anderson,R,ak,0.65,lower


In [550]:
## in the many to one merge we only select those cases where the first names dont match and assess them manually. 

look['check'] = np.where(look.first_name_x != look.first_name_y,1,0)
rectify_match = look[look.check == 1]

##save dataframe and edit manually 
# rectify_match.to_excel('./Data/assessed_match.xlsx')

#drop duplicates where there were correct matches 
correct_match = look[look.check == 0].drop_duplicates(subset = ['bill_document_first','first_name_x','last_name','first_name_y'], keep = 'first')

# call back edited dataframe
# manual_match = pd.read_excel('./Data/assessed_match.xlsx')

##Add back he matches into the original dataframe shape. Keep names I selected manually 
manual_match = manual_match[['state','session','bill_id','name_list', 'first_name_x', 'first_name_y', 'party', 'ideology', 'house']]
manual_match = pd.merge(manual_match,rectify_match[['state','session','bill_id','first_name_x','last_name']], on = ['state','session','bill_id','first_name_x'], how = 'left')
manual_match = manual_match[['first_name_x','first_name_y', 'last_name', 'party', 'ideology', 'house']]
rectify_match = pd.merge(rectify_match,manual_match, on = ['first_name_x','first_name_y','last_name'], how = 'left')
rectify_match = rectify_match[rectify_match.ideology_y.notna()]
rectify_match.shape
rectify_match = rectify_match.drop_duplicates(subset = ['bill_document_first','first_name_x','last_name','first_name_y'], keep = 'first')
rectify_match.shape
rectify_match = rectify_match[['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list', 'full_names', 'len_list', 'len_name', 'first_name_x',
       'last_name', 'first_name_y','party_y', 'ideology_y', 'house_y',
       'check']]
rectify_match.rename(columns = {'party_y':'party','ideology_y':'ideology','house_y':'house'}, inplace = True)

##matched full names 
full_name_matches = pd.concat([correct_match,rectify_match])
full_name_matches.shape


(206, 33)

(101, 33)

(302, 30)

In [561]:
## doing the same for bills where there is only first name 
slice_half = df_names.copy()
slice_half = slice_half[(slice_half['full_names'].isna()) | (slice_half['full_names'].apply(lambda x: x == []))]
slice_half['len_list'] = slice_half['name_list'].apply(lambda x: len(x))
slice_half = slice_half[slice_half.len_list == 1]
slice_half['len_name'] = slice_half['name_list'].apply(lambda x: len(x[0].split()))
slice_half = slice_half[slice_half.len_name < 2]

In [567]:

## only keep those rows which have unique matches. No way to know about those that have multiple matches
slice_half['last_name'] = slice_half.name_list.apply(lambda x: x[0])
look2 = pd.merge(slice_half, leg, on = ['last_name', 'state'], how = 'left')
look2.shape
keep_unique = pd.DataFrame(look2.last_name.value_counts().sort_values()).reset_index()
keep_unique_list = keep_unique[keep_unique.last_name == 1]['index'].tolist()
half_name_matches = look2[look2.last_name.isin(keep_unique_list)]
half_name_matches.shape

(1143, 28)

(298, 28)

In [569]:
## LegiScan matches
df_names2 = df_names.copy()
df_names2.shape
df_names2 = df_names2[(df_names2['full_names'].notna()) & (df_names2['full_names'].apply(lambda x: x != []))]
df_names2.shape
df_names2['len_full'] = df_names2.full_names.apply(lambda x: len(x))
df_names2 = df_names2[df_names2.len_full != 0]
df_names2.shape

(2585, 21)

(735, 21)

(735, 22)

In [572]:
def find_matching_elements(list1, list2):
    ''' ;LegiScan results in more sponsors than in raw bills dataset. From legiscan results 
    Keep only those names whose last name matches with the legislator name in the raw bill. No cases where 
    bill has two legislator with same names'''
    unique_words_list1 = set()
    # Split elements of list1 into words and add to the set
    for element in list1:
        parts = element.lower().split()
        for part in parts:
            unique_words_list1.add(part.replace(',', '').replace('.', '') )
    
    # Initialize set for unique matching names from list2
    unique_names_list2 = set()
    # Check if any part of the names from list2 matches any word in list1
    for full_name in list2:
        name_parts = full_name.lower().split()
        for name_part in name_parts:
            if name_part in unique_words_list1:
                unique_names_list2.add(full_name)  # Add original name to preserve formatting
                break  # Stop searching through this name as we have found a match
    
    return list(unique_names_list2)  #



In [573]:
df_names2['overlapping_names'] = df_names2.apply(lambda row: find_matching_elements(row['name_list'], row['full_names']), axis=1)
df_names2['len_full'] = df_names2.overlapping_names.apply(lambda x: len(x))
df_names2 = df_names2[df_names2.len_full != 0]

In [574]:
df_names2.shape

(673, 23)

In [575]:
## calling legislators again. different processing than we called last time

leg = pd.read_csv('./Data/legislators.csv')
leg['last_name'] = leg['last_name'].str.lower()
leg.head(2)

,first_name,last_name,party,state,u_id,ideology,senate2008,senate2009,senate2010,senate2011,senate2012,senate2013,senate2014,senate2015,house2008,house2009,house2010,house2011,id
0,Al,adams,D,ak,AK1995L036,-1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Tom,anderson,R,ak,AK2003L035,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [576]:
def get_ideology_scores(row):
    ''' For legiscan matched names merging with ideology dataset here seenms to complicate things. 
    instead look for the full name in the overlapping names column by first and last name in ideology and append the ideology score in a list.
    Append becaus there could be multiple sponsors. Later we will find the mean of each list for for each row  '''
    state = row['state']
    names = row['overlapping_names']
    scores = []

    for full_name in names:
        # Split the full name into first and last name and convert to lowercase
        parts = full_name.lower().split()
        if len(parts) >= 2:  # Check if the name can be split into at least first and last name
            first_name, last_name = parts[0], parts[-1]  # first and last name are the first and last parts

            # Filter  for matching first and last names and the same state
            matched = leg[(leg['first_name'].str.lower() == first_name) &
                                     (leg['last_name'].str.lower() == last_name) &
                                     (leg['state'].str.lower() == state)]
            scores.extend(matched['ideology'].tolist())  # Add matched ideology scores to the list

    return scores

# Apply function to each row in df to create the new 'ideology_scores' column

def get_party(row):
    ''' Same idea as above. bu this time get the party instead of ideology scores for all sponsors in a list and store as binary'''
    state = row['state']
    names = row['overlapping_names']
    values = []

    for full_name in names:
        # Split the full name into first and last name and convert to lowercase
        parts = full_name.lower().split()
        if len(parts) >= 2:  # Check if the name can be split into at least first and last name
            first_name, last_name = parts[0], parts[-1]  # Assuming first and last name are the first and last parts
            # Filter df_politicians for matching first and last names and the same state
            matched = leg[(leg['first_name'].str.lower() == first_name) &
                                     (leg['last_name'].str.lower() == last_name) &
                                     (leg['state'].str.lower() == state)]
            if not matched.empty:
                values.append(1 if matched['party'].values[0] == 'R' else 0)  # Add matched ideology scores to the list

    return values


def calculate_mean(row):
    ''' mean of each ideology score list for each row'''
    return np.mean(row) if len(row) > 0 else np.nan  

# Function to count the number of elements in the list
def count_elements(row):
    ''' number of sponsors for each bill'''
    return len(row)

# Function to calculate the variance (nanvar) of the list
def calculate_nanvar(row):
    '''Find the variance of the party sponsors. 1 for republican 0 for democrats '''
    return np.nanvar(row) if len(row) > 0 else np.nan  # Return NaN if the list is empty

# Function to determine party based on the first element
def determine_party(row):
    '''Also good to have party affiliation by party name along with binary'''
    if len(row) > 0:  # Check if the list is not empty
        return 'Republican' if row[0] == 1 else 'Democrat'
    else:
        return np.nan 



In [673]:
## apply all that. df_ide is legiscan matched bills
df_ide = df_names2.copy()
df_names2.columns
df_ide['ideology_scores'] = df_ide.apply(get_ideology_scores, axis=1)
df_ide['parties'] = df_ide.apply(get_party, axis=1)

Index(['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list', 'full_names', 'len_full', 'overlapping_names'],
      dtype='object')

In [674]:
## matches we found where raw bill had only last name. harmonizing the columns for readability
half_name_matches = half_name_matches[['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list', 'party', 'ideology']]

## matches we found where raw bill had full name. harmonizing the columns for readability

full_name_matches = full_name_matches[['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list', 'party', 'ideology']]

## matches we found whith manual matching. harmonizing the columns for readability

matched_manual = pd.concat([half_name_matches,full_name_matches])
matched_manual['ideology_scores'] = matched_manual['ideology'].apply(lambda x: [x])
matched_manual['parties'] = matched_manual['party'].apply(lambda x: [1 if x == 'R' else 0])


matched_manual = matched_manual[['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list','ideology_scores', 'parties']]

###All three above have single sponsors, so no need to apply average ideology score function on them or party finder.

##matched found with legiscan API. with the newly added ideology scores 
df_ide = df_ide[['date_signed', 'date_introduced', 'bill_document_first', 'date_updated',
       'short_title', 'bill_type', 'actions', 'summary', 'chamber', 'state',
       'session', 'action_dates', 'unique_id', 'bill_document_last',
       'date_created', 'bill_title', 'sponsers', 'bill_id', 'sunlight_id',
       'name_list','ideology_scores', 'parties']]

##create full data

all_match = pd.concat([matched_manual,df_ide])



In [730]:
all_match.shape

(1155, 7)

In [675]:

## get number of sponsors
all_match['len_full'] = all_match.ideology_scores.apply(lambda x: len(x))
all_match = all_match[all_match.len_full != 0]

#get mean of ideology scores in list because they have multiple sponsors
all_match['ideology'] = all_match.ideology_scores.apply(calculate_mean)

#how many sponsors
all_match['len_sponsors'] = all_match.ideology_scores.apply(count_elements)

#party variance for multiple sonsor bills. 0 when there is only 1 sponsor of course
all_match['party_variance'] = all_match.parties.apply(calculate_nanvar)

#get party name back from binary 
all_match['party'] = all_match.parties.apply(determine_party)



In [610]:
# all_match.to_excel('C:/Users/ayush/Downloads/ideology.xlsx', index = False)

In [647]:
##get back sampled alignments
alignments  = pd.read_excel('./Data/sampled.xlsx')

In [676]:
alignments.head(2)
all_match = all_match[['state','bill_id', 'unique_id','ideology','party','party_variance','len_sponsors']]
all_match['bill_id'] = all_match.bill_id.str.replace(' ','')
all_match.head(2)

,left_id,right_id,score,adjusted_alignment_score,left_bill_id,left_bill_state,right_bill_id,right_bill_state
0,nh_2011_HB186,ma_189th_H2875,39.0,23.9421,HB186,nh,H2875,ma
1,nh_2011_HB186,ny_2011-2012_S1300,16.0,15.0288,HB186,nh,S1300,ny


,state,bill_id,unique_id,ideology,party,party_variance,len_sponsors
0,wi,AJR5,wi_2013 Regular Session_AJR5,-1.059,Democrat,0.0,1
5,md,SB419,md_2008_SB419,0.282,Republican,0.0,1


In [677]:
all_match.shape

(1155, 7)

In [693]:
##first match bill ideology on left bills
alignment_ide = pd.merge(alignments,all_match, left_on = ['left_id'], right_on = ['unique_id'], how = 'left')

#drop all bills rows where left bill did not match. no point in keeping them since right bill is usefull only with left bill 
alignment_ide = alignment_ide[alignment_ide.bill_id.notna()]
alignment_ide.shape

#match on right bills
alignment_ide.rename(columns  ={'ideology':'ideology_left','party':'party_left','party_variance':'party_variance_left','len_sponsors':'len_sponsors_left'}, inplace = True)
alignment_ide = pd.merge(a,all_match, left_on = ['right_bill_id'], right_on = ['bill_id'], how = 'left')

#dropna again
alignment_ide = alignment_ide[alignment_ide.ideology.notna()]
alignment_ide.shape

#final columns we will use in analysis
alignment_ide.rename(columns  ={'ideology':'ideology_right','party':'party_right','party_variance':'party_variance_right','len_sponsors':'len_sponsors_right'}, inplace = True)

(674, 15)

(126, 22)

In [696]:
##also add a column for squared distance between bill pair sponsors
alignment_ide['dist'] = (alignment_ide['ideology_left'] - alignment_ide['ideology_right'])**2

In [ ]:
###save
alignment_ide.to_excel('./Data/joined_metadata.xlsx', index = False)